In [77]:
import numpy as np
from scipy.io import loadmat
from scipy.fftpack import fft
from scipy.signal import welch
import matplotlib.pyplot as plt
import pandas as pd
import seaborn

In [ ]:
## Dejar esto asi, para cuando haya que hacer un script que levante todo. Caro.
# pacientes = {'S': [], 'P': []}
# for c in (['S','P']):
#     for
# pacientes['P'].append(loadmat('P01.mat')['data'])
# print (pacientes['P'][0][0][0]) 

In [89]:
paciente = loadmat('P01.mat')['data']
## Creemos que las dimensiones son: impulso, electrodo, tiempo


In [110]:
ws = [[] for i in range(256)]
for epoch in paciente:
    for electrodo in epoch:
        w = welch(electrodo)
        ws[list(epoch).index(electrodo)].append(w[1])
promedios = [[] for i in range(256)]
for electrodo in ws:
    np.transpose(electrodo)
    promedios

/home/marto/.local/lib/python3.5/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 201, using nperseg = 201
  .format(nperseg, input_length))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()